# Transfer Learning VGG16 - 改 各种特征+size特征

参考：
+ [Exploration & Transforming Images in Python](https://www.kaggle.com/muonneutrino/exploration-transforming-images-in-python)
+ [Transfer Learning with VGG-16 CNN+AUG LB 0.1712](http://localhost:8888/notebooks/kaggle/ShipIceberg/TransferLearning-VGG.ipynb)
+ [Despeckling Synthetic Aperture Radar (SAR) Images](https://www.kaggle.com/jgroff/despeckling-synthetic-aperture-radar-sar-images)
+ [Submarineering. Size Matters](https://www.kaggle.com/submarineering/submarineering-size-matters)

In [1]:
#Mandatory imports
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit
from os.path import join as opj
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pylab
from scipy.ndimage.filters import uniform_filter
plt.rcParams['figure.figsize'] = 10, 10
%matplotlib inline

In [2]:
train = pd.read_json("../ShipIceberg/Data/train.json")
target_train=train['is_iceberg']

# 一直cannot resolve memory block
# test = pd.read_json("../ShipIceberg/Data/test.json") 

In [3]:
import json
with open('../ShipIceberg/Data/test.json', 'r') as f:
    test = json.load(f)
    test=pd.DataFrame(test)
    
print('done!')

done!


In [4]:
test['inc_angle'].shape

(8424,)

In [5]:
target_train=train['is_iceberg']
test['inc_angle']=pd.to_numeric(test['inc_angle'], errors='coerce')
train['inc_angle']=pd.to_numeric(train['inc_angle'], errors='coerce')#We have only 133 NAs.
train['inc_angle']=train['inc_angle'].fillna(method='pad')
X_angle=train['inc_angle']
# test['inc_angle']=pd.to_numeric(test['inc_angle'], errors='coerce')
train['inc_angle']=train['inc_angle'].fillna(method='pad')
X_test_angle=test['inc_angle']

In [ ]:
# implement functions to convert SAR data from decibel units to linear units and back again
def decibel_to_linear(band):
     # convert to linear units
    return np.power(10,np.array(band)/10)

def linear_to_decibel(band):
    return 10*np.log10(band)

# implement the Lee Filter for a band in an image already reshaped into the proper dimensions
def lee_filter(band, window, var_noise = 0.25):
        # band: SAR data to be despeckled (already reshaped into image dimensions)
        # window: descpeckling filter window (tuple)
        # default noise variance = 0.25
        # assumes noise mean = 0
    
        mean_window = uniform_filter(band, window)
        mean_sqr_window = uniform_filter(band**2, window)
        var_window = mean_sqr_window - mean_window**2

        weights = var_window / (var_window + var_noise)
        band_filtered = mean_window + weights*(band - mean_window)
        return band_filtered


In [7]:
# Isolation function.
def iso(arr):
    p = np.reshape(np.array(arr), [75,75]) >(np.mean(np.array(arr))+2*np.std(np.array(arr)))
    return p * np.reshape(np.array(arr), [75,75])

# Size in number of pixels of every isolated object.
def size(arr):     
    return np.sum(arr<-5)

# Feature engineering iso1 and iso2.
train['iso1'] = train.iloc[:, 0].apply(iso)
train['iso2'] = train.iloc[:, 1].apply(iso)

# train.head(5)

# Feature engineering s1 s2 and size.
train['s1'] = train.iloc[:,5].apply(size)
train['s2'] = train.iloc[:,6].apply(size)
train['size'] = train.s1+train.s2

X_size = train['size']

# Feature engineering iso1 and iso2.
test['iso1'] = test.iloc[:, 0].apply(iso)
test['iso2'] = test.iloc[:, 1].apply(iso)

# test.head(5)

test['s1'] = test.iloc[:,4].apply(size)
test['s2'] = test.iloc[:,5].apply(size)
test['size'] = test.s1+test.s2


X_test_size = test['size']

In [8]:
#Generate the training data
X_band_1=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_1"]])
X_band_2=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_2"]])

'''
# 需要循环？？？
# let's see if despeckling has any influence on the images
# plot nine different instances with different windows and noise levels (band 1)
windows = [2, 4, 8] # can be tuple too if not symetric
noise_var = np.array([1, 2, 4])

# 0-2
Wi = 0
Nj = 0

for i in range(0,1604):
    band_1_linear = decibel_to_linear(X_band_1[i])
    band_2_linear = decibel_to_linear(X_band_2[i])

    noise_var_1 = np.round(np.var(band_1_linear)*noise_var,10)
    noise_var_2 = np.round(np.var(band_2_linear)*noise_var,10)

    X_band_1[i] = linear_to_decibel(lee_filter(band_1_linear, windows[Wi], noise_var_1[Nj]))
    X_band_2[i] = linear_to_decibel(lee_filter(band_2_linear, windows[Wi], noise_var_2[Nj]))


# Sobel Operator & Averaging
from scipy import signal

Gx_sobel=np.array([[1,0,-1],[2,0,-2],[1,0,-1]])
Gy_sobel=np.array([[1,2,1],[0,0,0],[-1,-2,-1]])
lpf=1.0/25*np.ones((5,5))

for i in range(0,1604):
    
    img_1=X_band_1[i].copy()
    img_lpf_1=signal.convolve2d(img_1,lpf,mode='same')
    img_Gx_1=signal.convolve2d(img_lpf_1,Gx_sobel,mode='same')
    img_Gy_1=signal.convolve2d(img_lpf_1,Gy_sobel,mode='same')

    img_2=X_band_2[i].copy()
    img_lpf_2=signal.convolve2d(img_2,lpf,mode='same')
    img_Gx_2=signal.convolve2d(img_lpf_2,Gx_sobel,mode='same')
    img_Gy_2=signal.convolve2d(img_lpf_2,Gy_sobel,mode='same')

    X_band_1[i] = np.hypot(img_Gx_1,img_Gy_1)
    X_band_2[i] = np.hypot(img_Gx_2,img_Gy_2)
'''

X_band_3 = np.zeros((1604,75,75))
for i in range(0,1604):
    subt = abs(X_band_1[i]-X_band_2[i])
    W1 = subt/subt.max()
    W2=1-W1
    X_band_3[i]=W1 * X_band_1[i]+W2 * X_band_2[i]


# X_band_3=(X_band_1+X_band_2)/2  # 算术平均：尝试其他比例？
#X_band_3=X_band_1 * 0.5858 + X_band_2 * 0.4142
# 效果极差 X_band_3 = X_band_2 * X_band_2 / X_band_1
#X_band_3 = np.sqrt(X_band_1 * X_band_2)  # 调和·平均: sqrt invalid number???
#X_band_3 = X_band_1 / X_band_2
# X_band_3=X_band_2 * 2 - X_band_1

#from scipy import signal
#X_band_3=signal.fftconvolve(X_band_1, X_band_2, mode = 'same')

#X_band_3=np.array([np.full((75, 75), angel).astype(np.float32) for angel in train["inc_angle"]])

X_train = np.concatenate([X_band_1[:, :, :, np.newaxis]
                          , X_band_2[:, :, :, np.newaxis]
                         , X_band_3[:, :, :, np.newaxis]], axis=-1)


#Generate the test data
X_band_test_1=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test["band_1"]])
X_band_test_2=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test["band_2"]])

'''
for i in range(0,8424):
    band_test_1_linear = decibel_to_linear(X_band_test_1[i])
    band_test_2_linear = decibel_to_linear(X_band_test_2[i])

    noise_var_test_1 = np.round(np.var(band_test_1_linear)*noise_var,10)
    noise_var_test_2 = np.round(np.var(band_test_2_linear)*noise_var,10)

    X_band_test_1[i] = linear_to_decibel(lee_filter(band_test_1_linear, windows[Wi], noise_var_test_1[Nj]))
    X_band_test_2[i] = linear_to_decibel(lee_filter(band_test_2_linear, windows[Wi], noise_var_test_2[Nj]))


for i in range(0,8424):
    
    img_1=X_band_test_1[i].copy()
    img_lpf_1=signal.convolve2d(img_1,lpf,mode='same')
    img_Gx_1=signal.convolve2d(img_lpf_1,Gx_sobel,mode='same')
    img_Gy_1=signal.convolve2d(img_lpf_1,Gy_sobel,mode='same')

    img_2=X_band_test_2[i].copy()
    img_lpf_2=signal.convolve2d(img_2,lpf,mode='same')
    img_Gx_2=signal.convolve2d(img_lpf_2,Gx_sobel,mode='same')
    img_Gy_2=signal.convolve2d(img_lpf_2,Gy_sobel,mode='same')

    X_band_test_1[i] = np.hypot(img_Gx_1,img_Gy_1)
    X_band_test_2[i] = np.hypot(img_Gx_2,img_Gy_2)
'''

X_band_test_3 = np.zeros((8424,75,75))
for i in range(0,8424):
    subt = abs(X_band_test_1[i]-X_band_test_2[i])
    W1 = subt/subt.max()
    W2=1-W1
    X_band_test_3[i]=W1 * X_band_test_1[i]+W2 * X_band_test_2[i]

# X_band_test_3=(X_band_test_1+X_band_test_2)/2
#X_band_test_3=X_band_test_1 * 0.5858 + X_band_test_2 * 0.4142
#X_band_test_3 = X_band_test_2 * X_band_test_2 / X_band_test_1
#X_band_test_3 = np.sqrt(X_band_test_1*X_band_test_2)
#X_band_test_3=signal.fftconvolve(X_band_test_1, X_band_test_2, mode = 'same')
#X_band_test_3 = X_band_test_1 / X_band_test_2
# X_band_test_3=X_band_test_2 * 2 - X_band_test_1

#X_band_test_3=np.array([np.full((75, 75), angel).astype(np.float32) for angel in test["inc_angle"]])

X_test = np.concatenate([X_band_test_1[:, :, :, np.newaxis]
                          , X_band_test_2[:, :, :, np.newaxis]
                         , X_band_test_3[:, :, :, np.newaxis]], axis=-1)

# del train;del test;

数据处理：提取一些统计特征

In [9]:
def get_stats(data,label=1):
    data['max'+str(label)] = [np.max(np.array(x)) for x in data['band_'+str(label)] ]
    data['maxpos'+str(label)] = [np.argmax(np.array(x)) for x in data['band_'+str(label)] ]
    data['min'+str(label)] = [np.min(np.array(x)) for x in data['band_'+str(label)] ]
    data['minpos'+str(label)] = [np.argmin(np.array(x)) for x in data['band_'+str(label)] ]
    data['med'+str(label)] = [np.median(np.array(x)) for x in data['band_'+str(label)] ]
    data['std'+str(label)] = [np.std(np.array(x)) for x in data['band_'+str(label)] ]
    data['mean'+str(label)] = [np.mean(np.array(x)) for x in data['band_'+str(label)] ]
    data['p25_'+str(label)] = [np.sort(np.array(x))[int(0.25*75*75)] for x in data['band_'+str(label)] ]
    data['p75_'+str(label)] = [np.sort(np.array(x))[int(0.75*75*75)] for x in data['band_'+str(label)] ]
    data['mid50_'+str(label)] = data['p75_'+str(label)]-data['p25_'+str(label)]

    return data

train = get_stats(train,1)
train = get_stats(train,2)
X_min1 = train['min1']
X_max1 = train['max1']
X_std1 = train['std1']
X_med1 = train['med1']
X_mean1 = train['mean1']
X_min2 = train['min2']
X_max2 = train['max2']
X_std2 = train['std2']
X_med2 = train['med2']
X_mean2 = train['mean2']

test = get_stats(test,1)
test = get_stats(test,2)
X_test_min1 = test['min1']
X_test_max1 = test['max1']
X_test_std1 = test['std1']
X_test_med1 = test['med1']
X_test_mean1 = test['mean1']
X_test_min2 = test['min2']
X_test_max2 = test['max2']
X_test_std2 = test['std2']
X_test_med2 = test['med2']
X_test_mean2 = test['mean2']

In [10]:
#Import Keras.
from matplotlib import pyplot
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Input, Flatten, Activation
from keras.layers import GlobalMaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.merge import Concatenate
from keras.models import Model
from keras import initializers

from keras.optimizers import Adam
#from keras.optimizers import rmsprop
from keras.optimizers import SGD
from keras.optimizers import RMSprop
from keras.optimizers import Adagrad
from keras.optimizers import Adadelta
from keras.optimizers import Adamax
from keras.optimizers import Nadam


from keras.layers.advanced_activations import LeakyReLU, PReLU
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping
from keras.datasets import cifar10
from keras.applications.inception_v3 import InceptionV3
from keras.applications.vgg16 import VGG16
from keras.applications.xception import Xception
from keras.applications.mobilenet import MobileNet
from keras.applications.vgg19 import VGG19
from keras.layers import Concatenate, Dense, LSTM, Input, concatenate
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
# from keras.applications.vgg19 import preprocess_input
#from keras.applications.xception import preprocess_input
#from keras.applications.inception_v3 import preprocess_input
# ResNet

#Data Aug for multi-input
from keras.preprocessing.image import ImageDataGenerator

# 批处理大小的设置
# batch_size=64
batch_size=64

# Define the image transformations here
# 原始旋转范围10
#gen = ImageDataGenerator(horizontal_flip = True,
#                         vertical_flip = True,
#                         width_shift_range = 0.,
#                         height_shift_range = 0.,
#                         channel_shift_range=0,
#                         zoom_range = 0.2,
#                         rotation_range = 10)



# aug1
gen = ImageDataGenerator(horizontal_flip = True,
                         vertical_flip = True,
                         width_shift_range = 0.,
                         height_shift_range = 0.,
                         channel_shift_range=0.,
                         zoom_range = 0.2,
                         rotation_range = 10)


# Here is the function that merges our two generators
# We use the exact same generator with the same random seed for both the y and angle arrays
def gen_flow_for_thirteen_inputs(X1, X2, X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13, y):
    genX1 = gen.flow(X1,y,  batch_size=batch_size,seed=55)
    genX2 = gen.flow(X1,X2, batch_size=batch_size,seed=55)
    genX3 = gen.flow(X1,X3, batch_size=batch_size,seed=55)
    genX4 = gen.flow(X1,X4, batch_size=batch_size,seed=55)
    genX5 = gen.flow(X1,X5, batch_size=batch_size,seed=55)
    genX6 = gen.flow(X1,X6, batch_size=batch_size,seed=55)
    genX7 = gen.flow(X1,X7, batch_size=batch_size,seed=55)
    genX8 = gen.flow(X1,X8, batch_size=batch_size,seed=55)
    genX9 = gen.flow(X1,X9, batch_size=batch_size,seed=55)
    genX10 = gen.flow(X1,X10, batch_size=batch_size,seed=55)
    genX11 = gen.flow(X1,X11, batch_size=batch_size,seed=55)
    genX12 = gen.flow(X1,X12, batch_size=batch_size,seed=55)
    genX13 = gen.flow(X1,X13, batch_size=batch_size,seed=55)
    while True:
            X1i = genX1.next()
            X2i = genX2.next()
            X3i = genX3.next()
            X4i = genX4.next()
            X5i = genX5.next()
            X6i = genX6.next()
            X7i = genX7.next()
            X8i = genX8.next()
            X9i = genX9.next()
            X10i = genX10.next()
            X11i = genX11.next()
            X12i = genX12.next()
            X13i = genX13.next()
            #Assert arrays are equal - this was for peace of mind, but slows down training
            #np.testing.assert_array_equal(X1i[0],X2i[0])
            yield [X1i[0], X2i[1],X3i[1],X4i[1],X5i[1],X6i[1],X7i[1],
                  X8i[1],X9i[1],X10i[1],X11i[1],X12i[1],X13i[1]], X1i[1]

# Finally create generator
def get_callbacks(filepath, patience=2):
   es = EarlyStopping('val_loss', patience=10, mode="min")
   msave = ModelCheckpoint(filepath, save_best_only=True)
   return [es, msave]


def getVggAngleModel():
    input_2 = Input(shape=[1], name="angle")
    input_3 = Input(shape=[1], name="min1")
    input_4 = Input(shape=[1], name="max1")
    input_5 = Input(shape=[1], name="std1")
    input_6 = Input(shape=[1], name="med1")
    input_7 = Input(shape=[1], name="mean1")
    input_8 = Input(shape=[1], name="min2")
    input_9 = Input(shape=[1], name="max2")
    input_10 = Input(shape=[1], name="std2")
    input_11 = Input(shape=[1], name="med2")
    input_12 = Input(shape=[1], name="mean2")
    input_13 = Input(shape=[1], name="size")
    
    angle_layer = Dense(1, )(input_2)
    min1_layer = Dense(1, )(input_3)
    max1_layer = Dense(1, )(input_4)
    std1_layer = Dense(1, )(input_5)
    med1_layer = Dense(1, )(input_6)
    mean1_layer = Dense(1, )(input_7)
    min2_layer = Dense(1, )(input_8)
    max2_layer = Dense(1, )(input_9)
    std2_layer = Dense(1, )(input_10)
    med2_layer = Dense(1, )(input_11)
    mean2_layer = Dense(1, )(input_12)
    size_layer = Dense(1, )(input_13)
    
    # VGG16换成其他模型？？
    base_model = VGG16(weights='imagenet', include_top=False, 
                 input_shape=X_train.shape[1:], classes=1)
    x = base_model.get_layer('block5_pool').output
  
    '''
    # BatchNorm 未形成conv bn scale relu 的block
    input_tensor = Input(shape=X_train.shape[1:])
    bn = BatchNormalization()(input_tensor)
#     x = base_model(bn)
    x = base_model.get_layer('block5_pool')(bn)
'''
    x = GlobalMaxPooling2D()(x)
    merge_one = concatenate([x, angle_layer,
                             min1_layer,max1_layer,std1_layer,med1_layer,mean1_layer,
                            min2_layer,max2_layer,std2_layer,med2_layer,mean2_layer,size_layer])
    
#     merge_one = x
    merge_one = Dense(512, activation='relu', name='fc2')(merge_one)
#     merge_one = Dense(1024, activation='relu', name='fc2')(merge_one)
    merge_one = Dropout(0.3)(merge_one)
    merge_one = Dense(512, activation='relu', name='fc3')(merge_one)
    merge_one = Dropout(0.3)(merge_one)
    
    predictions = Dense(1, activation='sigmoid')(merge_one)
    
    model = Model(input=[base_model.input, input_2,input_3,input_4,input_5,input_6,
                        input_7,input_8,input_9,input_10,input_11,input_12,input_13], 
                  output=predictions)
#     model = Model(input=[base_model.input], output=predictions)
#     model = Model(input=[input_tensor, input_2], output=predictions)
    
    # 使用不同的优化
    sgd = SGD(lr=8e-4, decay=1e-6, momentum=0.9, nesterov=True)
    adagrad = Adagrad(lr = 1e-3, epsilon = 1e-6)
    rmsprop = RMSprop(lr=1e-3, rho = 0.9, epsilon=1e-6)
    adadelta = Adadelta(lr=1e-3, rho=0.95, epsilon=1e-06)
    adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
    adamax = Adamax(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
    nadam = Nadam(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=1e-08, schedule_decay=0.004)
    
    # 更换loss
    model.compile(loss='binary_crossentropy',
                  optimizer=sgd,
                  metrics=['accuracy'])
    
    return model

#Using K-fold Cross Validation with Data Augmentation.
def myAngleCV(X_train, X_angle, 
              X_min1,X_max1,X_std1,X_med1,X_mean1,
              X_min2,X_max2,X_std2,X_med2,X_mean2,X_size,
              X_test):
    # K-折交叉验证
    K=5
    
    folds = list(StratifiedKFold(n_splits=K, shuffle=True, random_state=16).split(X_train, target_train))
    y_test_pred_log = 0
    y_train_pred_log=0
    y_valid_pred_log = 0.0*target_train
    
    for j, (train_idx, test_idx) in enumerate(folds):
        print('\n===================FOLD=',j)
        X_train_cv = X_train[train_idx]
        y_train_cv = target_train[train_idx]
        X_holdout = X_train[test_idx]
        Y_holdout= target_train[test_idx]
        
        #Angle
        X_angle_cv=X_angle[train_idx]
        X_angle_hold=X_angle[test_idx]
        
        X_min1_cv=X_min1[train_idx]
        X_min1_hold=X_min1[test_idx]
        
        X_max1_cv=X_max1[train_idx]
        X_max1_hold=X_max1[test_idx]
        
        X_std1_cv=X_std1[train_idx]
        X_std1_hold=X_std1[test_idx]
        
        X_med1_cv=X_med1[train_idx]
        X_med1_hold=X_med1[test_idx]
        
        X_mean1_cv=X_mean1[train_idx]
        X_mean1_hold=X_mean1[test_idx]
        
        X_min2_cv=X_min2[train_idx]
        X_min2_hold=X_min2[test_idx]
        
        X_max2_cv=X_max2[train_idx]
        X_max2_hold=X_max2[test_idx]
        
        X_std2_cv=X_std2[train_idx]
        X_std2_hold=X_std2[test_idx]
        
        X_med2_cv=X_med2[train_idx]
        X_med2_hold=X_med2[test_idx]
        
        X_mean2_cv=X_mean2[train_idx]
        X_mean2_hold=X_mean2[test_idx]
        
        X_size_cv=X_size[train_idx]
        X_size_hold=X_size[test_idx]
        
        
        

        #define file path and get callbacks
        file_path = "./model/%s_aug_model_weights.hdf5"%j
        callbacks = get_callbacks(filepath=file_path, patience=5)
        gen_flow = gen_flow_for_thirteen_inputs(X_train_cv, X_angle_cv, 
                                              X_min1_cv,X_max1_cv,X_std1_cv,X_med1_cv,X_mean1_cv,
                                              X_min2_cv,X_max2_cv,X_std2_cv,X_med2_cv,X_mean2_cv,X_size_cv,
                                              y_train_cv)
        galaxyModel= getVggAngleModel()
        
        # 调整训练参数
        galaxyModel.fit_generator(
                gen_flow,
                steps_per_epoch=24,
                #steps_per_epoch=100,
                epochs=100,
                shuffle=True,
                verbose=1,
                validation_data=([X_holdout,X_angle_hold,
                                 X_min1_hold,X_max1_hold,X_std1_hold,X_med1_hold,X_mean1_hold,
                                 X_min2_hold,X_max2_hold,X_std2_hold,X_med2_hold,X_mean2_hold,X_size_hold], 
                                 Y_holdout),
                callbacks=callbacks)

        #Getting the Best Model
        galaxyModel.load_weights(filepath=file_path)
        #Getting Training Score
        score = galaxyModel.evaluate([X_train_cv,X_angle_cv,
                                     X_min1_cv,X_max1_cv,X_std1_cv,X_med1_cv,X_mean1_cv,
                                     X_min2_cv,X_max2_cv,X_std2_cv,X_med2_cv,X_mean2_cv,X_size_cv], 
                                     y_train_cv, verbose=0)
        print('Train loss:', score[0])
        print('Train accuracy:', score[1])
        #Getting Test Score
        score = galaxyModel.evaluate([X_holdout,X_angle_hold,
                                     X_min1_hold,X_max1_hold,X_std1_hold,X_med1_hold,X_mean1_hold,
                                     X_min2_hold,X_max2_hold,X_std2_hold,X_med2_hold,X_mean2_hold,X_size_hold], 
                                     Y_holdout, verbose=0)
        print('Test loss:', score[0])
        print('Test accuracy:', score[1])

        #Getting validation Score.
        pred_valid=galaxyModel.predict([X_holdout,X_angle_hold,
                                       X_min1_hold,X_max1_hold,X_std1_hold,X_med1_hold,X_mean1_hold,
                                       X_min2_hold,X_max2_hold,X_std2_hold,X_med2_hold,X_mean2_hold,X_size_hold])
        y_valid_pred_log[test_idx] = pred_valid.reshape(pred_valid.shape[0])

        #Getting Test Scores
        temp_test=galaxyModel.predict([X_test, X_test_angle,
                                      X_test_min1,X_test_max1,X_test_std1,X_test_med1,X_test_mean1,
                                      X_test_min2,X_test_max2,X_test_std2,X_test_med2,X_test_mean2,X_test_size])
        y_test_pred_log+=temp_test.reshape(temp_test.shape[0])

        #Getting Train Scores
        temp_train=galaxyModel.predict([X_train, X_angle,
                                       X_min1,X_max1,X_std1,X_med1,X_mean1,
                                       X_min2,X_max2,X_std2,X_med2,X_mean2,X_size])
        y_train_pred_log+=temp_train.reshape(temp_train.shape[0])

    y_test_pred_log=y_test_pred_log/K
    y_train_pred_log=y_train_pred_log/K

    print('\n Train Log Loss Validation= ',log_loss(target_train, y_train_pred_log))
    print(' Test Log Loss Validation= ',log_loss(target_train, y_valid_pred_log))
    return y_test_pred_log

Using TensorFlow backend.


In [11]:
preds=myAngleCV(X_train, X_angle, 
                X_min1,X_max1,X_std1,X_med1,X_mean1,
                X_min2,X_max2,X_std2,X_med2,X_mean2,X_size,
                X_test)


===================FOLD= 0


/home/jayden/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:165: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("de...)`


Epoch 1/100
24/24 [==============================] - 16s - loss: 0.9403 - acc: 0.6270 - val_loss: 0.3422 - val_acc: 0.8385
Epoch 2/100
24/24 [==============================] - 13s - loss: 0.6021 - acc: 0.7365 - val_loss: 0.5166 - val_acc: 0.7578
Epoch 3/100
24/24 [==============================] - 14s - loss: 0.5410 - acc: 0.7513 - val_loss: 0.3067 - val_acc: 0.8478
Epoch 4/100
24/24 [==============================] - 15s - loss: 0.4333 - acc: 0.8122 - val_loss: 0.2676 - val_acc: 0.8727
Epoch 5/100
24/24 [==============================] - 13s - loss: 0.6930 - acc: 0.6878 - val_loss: 0.4832 - val_acc: 0.7453
Epoch 6/100
24/24 [==============================] - 14s - loss: 0.4984 - acc: 0.7528 - val_loss: 0.3247 - val_acc: 0.8385
Epoch 7/100
24/24 [==============================] - 14s - loss: 0.4317 - acc: 0.7868 - val_loss: 1.6079 - val_acc: 0.5621
Epoch 8/100
24/24 [==============================] - 12s - loss: 0.5766 - acc: 0.7374 - val_loss: 0.4159 - val_acc: 0.7950
Epoch 9/100
24/2

24/24 [==============================] - 14s - loss: 0.2563 - acc: 0.8940 - val_loss: 0.2434 - val_acc: 0.9003
Epoch 18/100
24/24 [==============================] - 15s - loss: 0.2795 - acc: 0.8869 - val_loss: 0.1921 - val_acc: 0.9003
Epoch 19/100
24/24 [==============================] - 16s - loss: 0.2106 - acc: 0.9103 - val_loss: 0.2338 - val_acc: 0.9065
Epoch 20/100
24/24 [==============================] - 15s - loss: 0.2009 - acc: 0.9165 - val_loss: 0.1795 - val_acc: 0.9252
Epoch 21/100
24/24 [==============================] - 17s - loss: 0.1879 - acc: 0.9262 - val_loss: 0.2015 - val_acc: 0.9190
Epoch 22/100
24/24 [==============================] - 14s - loss: 0.1736 - acc: 0.9302 - val_loss: 0.1776 - val_acc: 0.9159
Epoch 23/100
24/24 [==============================] - 14s - loss: 0.2402 - acc: 0.8986 - val_loss: 0.2959 - val_acc: 0.8474
Epoch 24/100
24/24 [==============================] - 15s - loss: 0.2692 - acc: 0.8754 - val_loss: 0.2139 - val_acc: 0.9097
Epoch 25/100
24/24 [=

24/24 [==============================] - 14s - loss: 0.2462 - acc: 0.9059 - val_loss: 0.2690 - val_acc: 0.8844
Epoch 16/100
24/24 [==============================] - 14s - loss: 0.2244 - acc: 0.9098 - val_loss: 0.2533 - val_acc: 0.8906
Epoch 17/100
24/24 [==============================] - 15s - loss: 0.1980 - acc: 0.9146 - val_loss: 0.2532 - val_acc: 0.8938
Epoch 18/100
24/24 [==============================] - 14s - loss: 0.1790 - acc: 0.9276 - val_loss: 0.2618 - val_acc: 0.8969
Epoch 19/100
24/24 [==============================] - 16s - loss: 0.1939 - acc: 0.9217 - val_loss: 0.2666 - val_acc: 0.8875
Epoch 20/100
24/24 [==============================] - 15s - loss: 0.2689 - acc: 0.8909 - val_loss: 0.2507 - val_acc: 0.9062
Epoch 21/100
24/24 [==============================] - 13s - loss: 0.2469 - acc: 0.9181 - val_loss: 1.3434 - val_acc: 0.6312
Epoch 22/100
24/24 [==============================] - 16s - loss: 0.3347 - acc: 0.8741 - val_loss: 0.2854 - val_acc: 0.8750
Epoch 23/100
24/24 [=

In [12]:
#Submission for each day.
submission = pd.DataFrame()
submission['id']=test['id']
submission['is_iceberg']=preds
submission.to_csv('subVGG.csv', index=False)